In [ ]:
import json
import re
import numpy as np
import os
import glob
import sklearn as sk
from sklearn.model_selection import train_test_split
import shutil

In [ ]:
def get_txt_annotation(path_to_file, path_to_txt_folder, filename):
  
  '''
Create a txt file with normalized coordinates of single-class polygon annotations from geojson
Создаёт текстовый файл с нормализованными координатами полигональных аннотаций единственного класса из файл формата geojson

  path_to_file: path to geojson/json file with img annotations
  filename: name of existing/new txt file used as final txt file with img annotations
  '''
  coords_list = []
  str_coords_list = []
  f = open(os.path.join(path_to_txt_folder, filename),'w')
  with open(path_to_file) as file:
        j_file = json.load(file)

  for dict_ in j_file:
        geometry = dict_['geometry']
        coords = geometry['coordinates']
        coords_list.append(coords)

  for coord in coords_list:
        coord = str(coord)
        str_coord = re.sub('[^\d\.]',' ', coord)
        str_coords_list.append(str_coord)

  for s in str_coords_list:
        # получить списки с координатами масок на изображении внутри цикла for
        a = [float(x) for x in s.split()]
        # создадим список с нормализованными значениями
        norm_list = []
        for coord in a:
            if a.index(coord) % 2:
                norm_list.append(coord/942)
            else:
                norm_list.append(coord/1716)

        # удалить символы запятых и квадратных скобок
        str_a = ' '.join(map(str,norm_list))
        # добавить метку единственного класса 0
        str_a = '0 ' + str_a
        #записать в txt файл каждую строку с новой строки
        f.write(str_a + "\n")

In [ ]:
path_to_folder = 'synovial_annots_json/*'
path_to_txt_folder = 'txt_annots'

In [16]:
for path_to_file in glob.glob(path_to_folder):
    txt_filename = os.path.basename(path_to_file).rstrip('json') + 'txt'
    try:
        get_txt_annotation(path_to_file, path_to_txt_folder, txt_filename)
        pass
    except:
        continue

In [17]:
img_names = [] # create a list of file names

for file in glob.glob(path_to_txt_folder + '/*'):
    file_name = os.path.basename(file)
    img_names.append(file_name)

In [ ]:
# create YAML-format dirs
os.makedirs("yolo_v8/valid/labels")
os.makedirs("yolo_v8/train/labels")
os.makedirs("yolo_v8/valid/images")
os.makedirs("yolo_v8/train/images")

In [18]:
# split file names to train and valid 
X_train, X_val, y_train, y_val = sk.model_selection.train_test_split(img_names, img_names, train_size=0.8, random_state=42)

In [19]:
path_to_img = 'synovial_images'

In [20]:
#val_labels, images
for txt_id in X_val:
    shutil.move(os.path.join(path_to_txt_folder, txt_id), 'yolo_v8/valid/labels')
    shutil.move(os.path.join(path_to_img, txt_id.rstrip('.txt') + '.tif'), 'yolo_v8/valid/images')
    

#train_labels, images
for txt_id in X_train:
    shutil.move(os.path.join(path_to_txt_folder, txt_id), 'yolo_v8/train/labels')
    shutil.move(os.path.join(path_to_img, txt_id.rstrip('.txt') + '.tif'), 'yolo_v8/train/images')

In [2]:
pip install git+https://github.com/ultralytics/ultralytics.git@main

  Cloning https://github.com/ultralytics/ultralytics.git (to revision main) to /tmp/pip-req-build-kb18i6ow
  Running command git clone --filter=blob:none --quiet https://github.com/ultralytics/ultralytics.git /tmp/pip-req-build-kb18i6ow
  Resolved https://github.com/ultralytics/ultralytics.git to commit 9df18ca9eff114743e3752ff95fc9c35d8e1202b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for ultralytics: filename=ultralytics-8.2.64-py3-none-any.whl size=825491 sha256=11fbb5d42c16425589b37e3970b5ad6ad144ff26cb5743363dab0984903391d1
  Stored in directory: /tmp/pip-ephem-wheel-cache-e_m7nhwx/wheels/60/db/e5/6abbdd13b4e3d5e2fdcd87690cd8738f842eeb37142fdccb5b
Successfully built ultralytics
  Attempting uninstall: ultralytics
    Found existing installation: ultralytics 8.2.62
    Uninstalling ultralytics-8.2.62:
      Successfully uninstalled ultralytics-8.2.62
  Consider adding this di

In [1]:
from ultralytics import YOLO

ImportError: libGL.so.1: cannot open shared object file: No such file or directory

In [3]:
# to fix error
! pip install opencv-python-headless
import cv2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 19.0 MB/s eta 0:00:00:00:0100:01


In [4]:
from ultralytics import YOLO

In [5]:
seg_s_model = YOLO('YOLOv8s-seg.pt')

In [6]:
seg_s_model.train(data='config.yaml', degrees = 360, flipud = 0.2, fliplr = 0.2, epochs=100)

Ultralytics YOLOv8.2.64 🚀 Python-3.10.13 torch-2.3.1+cpu CPU (AMD EPYC 7763 64-Core Processor)
engine/trainer: task=segment, mode=train, model=YOLOv8s-seg.pt, data=config.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_b

RuntimeError: Dataset 'config.yaml' error ❌ 
Dataset 'config.yaml' images not found ⚠️, missing path '/yolo_v8/valid/images'
Note dataset download directory is '/workspaces/YOLOV8_seg_synovial_img/datasets'. You can update this in '/home/codespace/.config/Ultralytics/settings.yaml'